In [ ]:
import pandas as pd
import numpy as np
from osgeo import gdal
from osgeo import osr

In [ ]:
tuples = {}
for var_num in range(1, 20):
    print(var_num)
    file_path = "wc2.1_10m_bio/wc2.1_10m_bio_" + str(var_num) + ".tif"
    # Open the file
    src = gdal.Open(file_path)

    # Get the pixel values as a numpy array
    data = src.ReadAsArray()

    # Get the geotransform (mapping from pixel coordinates to geographic coordinates)
    geotransform = src.GetGeoTransform()

    # Get the spatial reference system (SRS) of the raster
    srs = src.GetProjection()

    # Define the source CRS
    src_crs = osr.SpatialReference()
    src_crs.ImportFromWkt(src.GetProjection())
    #print(src_crs)

    # Define the target CRS (WGS84)
    target_crs = osr.SpatialReference()
    target_crs.SetWellKnownGeogCS("WGS84")
    #print(target_crs)

    # Create a coordinate transformation object
    transform = osr.CoordinateTransformation(src_crs, target_crs)

    # Get the coordinates of the pixels
    cols, rows = data.shape[1], data.shape[0]
    trash_val = data.min()
    for y in range(rows):
        for x in range(cols):
            # Convert pixel coordinates to geographic coordinates
            if data[y,x] != trash_val:
                lon_, lat_ = gdal.ApplyGeoTransform(geotransform, x, y)
                lon_, lat_, z = transform.TransformPoint(lon_, lat_)
                if (lon_, lat_) not in tuples:
                    tuples[(lon_, lat_)] = {var_num: data[y,x]}
                else:
                    tuples[(lon_, lat_)][var_num] = data[y,x]

In [ ]:
dataset = []
for key in tuples:
    new_sample = {'lon': key[0], 'lat': key[1]}
    for i in tuples[key]:
        new_sample['var_' + str(i)] = tuples[key][i]
    dataset.append(new_sample)

In [ ]:
df = pd.DataFrame(dataset)
df.to_csv('all_clim_data.csv', index=False)

In [ ]:
background_df = df.copy()
background_df['Species'] = 'background'
old_cols = ['Species', 'lon', 'lat'] + ['var_' + str(i) for i in range(1, 20)]
cols = ['Species', 'X', 'Y'] + ['Var' + str(i) for i in range(1, 20)]
background_df[cols] = background_df[old_cols]
background_df = background_df.drop(columns=old_cols)
background_df['Species'] = 'background'
background_df.to_csv('background.csv', index=False)